In [10]:
import numpy as np
import pandas as pd
from PIL import Image
import sys
import time
from csv import writer
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

In [3]:
def confuse(y_true, y_pred):
    return pd.DataFrame(
        data = confusion_matrix(y_true, y_pred).T,
        columns = ['real false', 'real true'],
        index = ['pred false', 'pred true']
    )

In [4]:
def evl(model, y_true, y_pred, title):
    matrix = confusion_matrix(y_true, y_pred).T
    tp = matrix[1, 1]
    tn = matrix[0, 0]
    fp = matrix[1, 0]
    fn = matrix[0, 1]
    return pd.DataFrame(
        data = {
            'accuracy': [np.round((tp + tn) / (tp + tn + fp + fn), 3)],
            'sensitivity': [np.round(tp / (tp + fn), 3)],
            'specificity': [np.round(tn / (tn + fp), 3)],
            'precision': [np.round(tp / (tp + fp), 3)],
            'ROC AUC': [np.round(roc_auc_score(y_true, y_pred), 3)],
        },
        index = [title]
    )

### Getting raw image pixel data
When opening an image with the PIL library, the pixel data can be extracted with the `getdata()` method. The images were saved as 32x32 greyscale jpeg images in the `binary_images/img/` folder. Greyscale means that every pixel is described by a single integer between 0 and 255, where 0 is pitch black and 255 is perfect white.

In [12]:
img = Image.open('../binary_images/img/img_0_0.jpg')

In [13]:
data = list(img.getdata())

In [14]:
data

[89,
 90,
 92,
 92,
 90,
 87,
 83,
 80,
 76,
 77,
 78,
 80,
 82,
 83,
 85,
 85,
 84,
 84,
 84,
 84,
 84,
 84,
 84,
 84,
 83,
 83,
 84,
 85,
 85,
 84,
 83,
 82,
 89,
 91,
 92,
 92,
 90,
 87,
 83,
 81,
 77,
 78,
 79,
 80,
 82,
 84,
 85,
 86,
 84,
 84,
 84,
 84,
 84,
 84,
 84,
 84,
 86,
 86,
 85,
 85,
 85,
 86,
 87,
 87,
 90,
 91,
 92,
 92,
 90,
 87,
 84,
 82,
 78,
 78,
 80,
 81,
 83,
 84,
 85,
 86,
 84,
 84,
 84,
 84,
 84,
 84,
 84,
 84,
 87,
 86,
 85,
 84,
 85,
 87,
 88,
 90,
 90,
 91,
 92,
 91,
 90,
 88,
 85,
 83,
 79,
 80,
 81,
 82,
 83,
 85,
 85,
 86,
 85,
 85,
 85,
 85,
 85,
 85,
 85,
 85,
 84,
 84,
 85,
 85,
 86,
 87,
 87,
 88,
 91,
 91,
 91,
 91,
 90,
 88,
 86,
 85,
 81,
 81,
 82,
 83,
 84,
 85,
 86,
 86,
 85,
 85,
 85,
 85,
 85,
 85,
 85,
 85,
 83,
 85,
 87,
 88,
 89,
 88,
 87,
 86,
 91,
 91,
 91,
 91,
 90,
 89,
 87,
 87,
 82,
 82,
 83,
 84,
 85,
 85,
 86,
 86,
 86,
 86,
 86,
 86,
 86,
 86,
 86,
 86,
 86,
 87,
 88,
 89,
 89,
 88,
 87,
 86,
 91,
 91,
 91,
 91,
 90,
 89,
 88,
 88,


In [2]:
32*32

1024

In [15]:
len(data)

1024

In [17]:
32 * 32 # size of image

1024

### Manually label flood and non-flood images.
In order to train a model on whether or not an image is flooded, it needs both images of floods and images of non-floods. The images included in the object detection portion of the lab were all of floods, so I added an extra ~400 pictures that I had manually filtered out of my NYT query. Below is the list of manually labeled data, where every number below corresponds to the index of one non-flood image in the dataset. 

In [ ]:
with open('../binary_images/data/non_flood_images.txt') as file:
    file_contents = file.readlines()

non_flood_images =[int(num.strip()) for num in (''.join(file_contents)).split(',')]

In [6]:
len(non_flood_images)

365

In [1]:
365 / 908

0.40198237885462557

### Processing data

The 145,000 images from data augmentation is about 1 GB worth of memory. During programming I ran into many performance drops because of the large amount of data, in particular with creating DataFrames. The way that worked the fastest was saving the entire data of every image into a single csv file, which is then loaded into a dataframe below. The images are saved to `binary_images/data/binary_img.csv`. Note that even though the data only takes up 1 GB on its own, training the models and some additional overhead push the memory usage up to 2 GB or higher. As a result the code below will likely struggle or fail when run on machines with only 4 GB of memory.

In [24]:
start_time = time.time()
label_list = []
img_list = []
for i in range(0, 908):
    if i % 100 == 0:
        print(f'Loading image {i} at time {round(time.time() - start_time)} s...')
    is_flood = int(i not in non_flood_images)
    for j in range(0, 160):
        with Image.open(f'../binary_images/img/img_{i}_{j}.jpg') as image:
            img_list.append(np.array(image.getdata(), np.uint8))
            label_list.append(is_flood)

with open('../binary_images/data/binary_img.csv', 'w') as file:
    csv_writer = writer(file)
    
    for i, image in enumerate(img_list):
        if i % 10000 == 0:
            print(f'Writing line {i} at time {round(time.time() - start_time)} s...')
        csv_writer.writerow(list(image) + [label_list[i]])

print(f'Done. Took {round(time.time() - start_time)} seconds.')
del label_list
del img_list


Loading image 0 at time 0 s...
Loading image 100 at time 5 s...
Loading image 200 at time 9 s...
Loading image 300 at time 13 s...
Loading image 400 at time 17 s...
Loading image 500 at time 21 s...
Loading image 600 at time 26 s...
Loading image 700 at time 30 s...
Loading image 800 at time 34 s...
Loading image 900 at time 39 s...
Writing line 0 at time 39 s...
Writing line 10000 at time 43 s...
Writing line 20000 at time 48 s...
Writing line 30000 at time 52 s...
Writing line 40000 at time 56 s...
Writing line 50000 at time 60 s...
Writing line 60000 at time 64 s...
Writing line 70000 at time 69 s...
Writing line 80000 at time 73 s...
Writing line 90000 at time 77 s...
Writing line 100000 at time 82 s...
Writing line 110000 at time 86 s...
Writing line 120000 at time 90 s...
Writing line 130000 at time 95 s...
Writing line 140000 at time 99 s...
Done. Took 102 seconds.


### Load the image data into a dataframe
Now that all the images are saved into one file, Pandas can take over and load it with great speed. This method has some notable drawbacks. A computer is limited by its RAM in loading the size of this file, meaning the file should never realistically go above 4-6 GB. Not to mention that this will more than double the size of the image data on your hard drive, since jpg images are compressed while the csv result will not be. This solution only works for datasets on the order of this size or less.

In [7]:
%%time
df = pd.read_csv('../binary_images/data/binary_img.csv', header=None)

CPU times: user 11.9 s, sys: 1.55 s, total: 13.5 s
Wall time: 14.3 s


In [8]:
sys.getsizeof(df) / 1_000_000_000 # size of dataframe in gb

1.19129616

In [4]:
df.shape # (number of images, pixels per image + label)

(145280, 1025)

In [9]:
df # column 1024 is the label column

0     1     2     3     4     5     6     7     8     9     ...  1015  \
0         89    90    92    92    90    87    83    80    76    77  ...    88   
1         88    88    88    88    88    88    88    88    90    89  ...    95   
2         93    91    89    90    93    95    93    91    94    93  ...   113   
3         89    92    90    85    84    88    93    93    86    85  ...    52   
4         84    84    84    84    84    84    84    84    83    83  ...    70   
...      ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
145275   164   175   146   189   240   214   142   166   151   152  ...   176   
145276   227   227   226   226   226   225   225   225   224   224  ...    75   
145277   107    99   201   149    52    67    42   152   100    45  ...   168   
145278   195   253   252   255   235   227   216   228   132   108  ...    18   
145279   246   255   255   250   251   255   197   189   184   202  ...   153   

        1016  1017  1018  1019  1020  1021  1022  1023  1024  
0         75    72    69    66    65    67    71    73     1  
1         95   102   102    94    94   102   105   101     1  
2         84    65    70    81    76    79    82    67     1  
3         47    66    82    83    82    85    84    79     1  
4         82    86    87    84    77    71    65    61     1  
...      ...   ...   ...   ...   ...   ...   ...   ...   ...  
145275   133    87   146   250   246   255   236   189     0  
145276    85    77    83    93    96   108   127   138     0  
145277   172   173   173   171   168   165   164   165     0  
145278    21    23    25    24    23    23    26    28     0  
145279   146   153   150   141   140   150   154   150     0  

[145280 rows x 1025 columns]

In [6]:
# class balance
df[1024].mean()

0.5980176211453745

#### Logistic model

In [12]:
%%time
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=[1024]), df[1024], test_size = 0.25, random_state = 2, stratify = df[1024])
del df

lr = LogisticRegression(max_iter = 100)
lr.fit(X_train, y_train)
print('train', lr.score(X_train, y_train))
print('test', lr.score(X_test, y_test))

/home/eeyle/anaconda3/envs/dsi/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


train 0.6066905286343612
test 0.5972191629955947
CPU times: user 53.3 s, sys: 928 ms, total: 54.2 s
Wall time: 15.4 s


The logistic regression model performs no better than the baseline model.

In [6]:
y_test.mean()

0.5980176211453745

In [9]:
pd.DataFrame(
    data = confusion_matrix(y_test, lr.predict(X_test)).T,
    columns = ['real false', 'real true'],
    index = ['pred false', 'pred true']
)

real false  real true
pred false        2681       2710
pred true        11919      19010

#### SVM

In [3]:
%%time
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=[1024]), df[1024], test_size = 0.25, random_state = 2)
del df

svc = SVC(kernel = 'rbf', max_iter = 1000)
svc.fit(X_train, y_train)
print('train', svc.score(X_train, y_train))
print('test', svc.score(X_test, y_test))

/home/eeyle/anaconda3/envs/dsi/lib/python3.7/site-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


train 0.4491831864904552
test 0.44812775330396476
CPU times: user 9min 48s, sys: 619 ms, total: 9min 48s
Wall time: 9min 48s


An SVM classifier with RBF kernel performs considerably worse than the baseline model.

#### Tree-based model

In [15]:
%%time
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=[1024]), df[1024], test_size = 0.25, random_state = 2)
del df

forest = RandomForestClassifier(
    n_estimators = 1024,
    max_depth = 32,
    n_jobs = 7
)
forest.fit(X_train, y_train)
print('train', forest.score(X_train, y_train))
print('test', forest.score(X_test, y_test))

train 0.9971732745961821
test 0.7032764317180616
CPU times: user 1h 13min 9s, sys: 3.11 s, total: 1h 13min 12s
Wall time: 10min 31s


In [16]:
confuse(y_test, forest.predict(X_test))

real false  real true
pred false        4487        691
pred true        10086      21056

In [20]:
evl(forest, y_test, forest.predict(X_test), 'Forest')

accuracy  sensitivity  specificity  precision  ROC AUC
Forest     0.703        0.968        0.308      0.676    0.638

The random forest model performs quite well. Stumps (max_depth ~ 1-4) perform poorly, while going much deeper than this results in perfect overfitting on the train dataset. The overfitting can be managed by a huge number of estimators.

#### Neural network

https://scikit-learn.org/stable/modules/neural_networks_supervised.html

In [4]:
%%time
df = pd.read_csv('../binary_images/data/binary_img.csv', header=None)
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns=[1024]), 
    df[1024], 
    test_size = 0.25, 
    random_state = 2
)
del df # going above your system's allocated memory will instantly crash the ipynb kernel, so just reload and delete the csv every time

ss = StandardScaler(copy = False, with_mean = 0, with_std = 1)
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

mlp = MLPClassifier(
    hidden_layer_sizes = (256, 64, 16, 4),
    activation = 'logistic',
    solver = 'adam',
    random_state = 2,
)    

mlp.fit(X_train, y_train)
print('train', mlp.score(X_train, y_train))
print('test', mlp.score(X_test, y_test))

/home/eeyle/anaconda3/envs/dsi/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


train 0.8320851688693098
test 0.7987885462555067
CPU times: user 57min 37s, sys: 55.5 s, total: 58min 33s
Wall time: 15min 11s


In [10]:
confuse(y_test, mlp.predict(X_test))

real false  real true
pred false       11946       4681
pred true         2627      17066

In [7]:
evl(mlp, y_test, mlp.predict(X_test), 'MLP')

accuracy  sensitivity  specificity  precision  ROC AUC
MLP     0.799        0.785         0.82      0.867    0.802

A simple MLP NN performs extraordinarily well at the cost of a ridiculous training time.

#### More neural network models

In [10]:
def nn_model(layers):
    start_time = time.time()
    df = pd.read_csv('../binary_images/data/binary_img.csv', header=None)
    X_train, X_test, y_train, y_test = train_test_split(
        df.drop(columns=[1024]), 
        df[1024], 
        test_size = 0.25, 
        random_state = 2
    )
    del df # going above your system's allocated memory will instantly crash the ipynb kernel, so just reload and delete the csv every time

    ss = StandardScaler(copy = False, with_mean = 0, with_std = 1)
    X_train = ss.fit_transform(X_train)
    X_test = ss.transform(X_test)

    mlp = MLPClassifier(
        hidden_layer_sizes = layers, 
        activation = 'logistic',
        solver = 'adam',
        random_state = 2,
    )    

    mlp.fit(X_train, y_train)
    print('train', mlp.score(X_train, y_train))
    print('test', mlp.score(X_test, y_test))
    print(f'Took {round(time.time() - start_time)} seconds.')

In [8]:
nn_model((16, 4))

/home/eeyle/anaconda3/envs/dsi/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


train 0.7468704111600587
test 0.7112334801762115


In [15]:
def nn_count(layers):
    print('n nodes', sum(layers))
    print('n connections', sum([layers[i] * layers[i+1] for i in range(len(layers)-1)]))

In [16]:
nn_count((64, 16, 4))

n nodes 84
n connections 1088


In [18]:
nn_model((64, 16, 4))

/home/eeyle/anaconda3/envs/dsi/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


train 0.8230543318649045
test 0.7816629955947136
Took 255 seconds.


In [25]:
nn_count((32, 16, 8))

n nodes 56
n connections 640


In [26]:
nn_model((32, 16, 8))

/home/eeyle/anaconda3/envs/dsi/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


train 0.751642804698972
test 0.7254955947136564
Took 177 seconds.


In [27]:
nn_count((128, 8))

n nodes 136
n connections 1024


In [28]:
nn_model((128, 8))

/home/eeyle/anaconda3/envs/dsi/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


train 0.846264684287812
test 0.8210352422907489
Took 442 seconds.


In [41]:
nn_count((128, 32, 8))

n nodes 168
n connections 4352


In [42]:
nn_model((128, 32, 8))

/home/eeyle/anaconda3/envs/dsi/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


train 0.8215859030837004
test 0.7878579295154186
Took 457 seconds.


In [55]:
nn_count((64, 128, 32))

n nodes 224
n connections 12288


In [56]:
nn_model((64, 128, 32))

/home/eeyle/anaconda3/envs/dsi/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


train 0.8028267254038179
test 0.7777533039647577
Took 337 seconds.


In [57]:
nn_count((64, 64, 64, 64))

n nodes 256
n connections 12288


In [58]:
nn_model((64, 64, 64, 64))

/home/eeyle/anaconda3/envs/dsi/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


train 0.793906020558003
test 0.7664922907488987
Took 346 seconds.


### Final model

In [8]:
%%time
df = pd.read_csv('../binary_images/data/binary_img.csv', header=None)
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns=[1024]), 
    df[1024], 
    test_size = 0.25, 
    random_state = 2
)
del df # going above your system's allocated memory will instantly crash the ipynb kernel, so just reload and delete the csv every time

ss = StandardScaler(copy = False, with_mean = 0, with_std = 1)
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

mlp = MLPClassifier(
    hidden_layer_sizes = (128, 16), 
    activation = 'logistic',
    solver = 'adam',
    random_state = 2,
    max_iter = 500
)    

mlp.fit(X_train, y_train)
print('train', mlp.score(X_train, y_train))
print('test', mlp.score(X_test, y_test))

train 0.8584342878120411
test 0.8239262114537445
CPU times: user 24min 21s, sys: 30.6 s, total: 24min 52s
Wall time: 6min 46s


In [11]:
confuse(y_test, mlp.predict(X_test))

real false  real true
pred false       12418       4240
pred true         2155      17507

In [12]:
evl(mlp, y_test, mlp.predict(X_test), 'MLP')

accuracy  sensitivity  specificity  precision  ROC AUC
MLP     0.824        0.805        0.852       0.89    0.829

In [13]:
final_model_path = '../binary_images/flood_image_classifier_mlp_128_16.sav'
pickle.dump(mlp, open(final_model_path, 'wb'))